# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.111196e+02     1.484812e+00
 * time: 0.07368183135986328
     1     1.136166e+01     9.065097e-01
 * time: 1.4496738910675049
     2    -1.137165e+01     1.025719e+00
 * time: 1.5349669456481934
     3    -3.387718e+01     7.177525e-01
 * time: 1.6498589515686035
     4    -4.739144e+01     5.533646e-01
 * time: 1.7811799049377441
     5    -5.689811e+01     2.142668e-01
 * time: 1.8838248252868652
     6    -5.982392e+01     1.276920e-01
 * time: 1.9609968662261963
     7    -6.091988e+01     6.044469e-02
 * time: 2.0345218181610107
     8    -6.130942e+01     7.088004e-02
 * time: 2.1386940479278564
     9    -6.159165e+01     4.432823e-02
 * time: 2.215906858444214
    10    -6.178953e+01     3.363145e-02
 * time: 2.2921929359436035
    11    -6.195361e+01     2.054222e-02
 * time: 2.3701369762420654
    12    -6.202657e+01     2.093181e-02
 * time: 2.4503068923950195
    13    -6.210541e+01     1.703304e-02
 * time: 2.5299389362

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557655
    AtomicNonlocal      14.8522630
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485365 
    Xc                  -19.3336818

    total               -62.261666459150
